# Section 2
* All of the sudden, the graphs don't work when concatting them together. 
* Start a fresh nb to debug.

In [1]:
import _report_utils
import _section2_utils as section2
import altair as alt
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS, SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")
import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [4]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

## Data

In [5]:
org_name = "City and County of San Francisco"

In [6]:
df = section2.load_schedule_vp_metrics(org_name)

In [15]:
df.Period.value_counts()

all_day    1623
offpeak    1570
peak       1461
Name: Period, dtype: int64

In [7]:
all_day = df.loc[df["Period"] == "all_day"]

In [8]:
timeliness_df = section2.timeliness_trips(df)

In [9]:
#section2.filtered_route(df)

## Charts

### Text

In [12]:
most_recent_date = df["Date"].max()
route_merge_cols = ["Route", "Direction"]

all_day_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "all_day")][
        route_merge_cols
        + [
            "Average Scheduled Service (trip minutes)",
            "Average Stop Distance (miles)",
            "# scheduled trips",
            "GTFS Availability",
        ]
    ]

In [14]:
all_day_stats.head()

,Route,Direction,Average Scheduled Service (trip minutes),Average Stop Distance (miles),# scheduled trips,GTFS Availability
11,1 CALIFORNIA,0.00,37.99,0.11,175,schedule_and_vp
47,1 CALIFORNIA,1.00,36.35,0.10,178,schedule_and_vp
83,12 FOLSOM-PACIFIC,0.00,38.59,0.11,85,schedule_and_vp
119,12 FOLSOM-PACIFIC,1.00,42.31,0.12,85,schedule_and_vp
155,14 MISSION,0.00,47.98,0.16,156,schedule_and_vp


In [ ]:
def route_stats(df: pd.DataFrame) -> pd.DataFrame:
    most_recent_date = df["Date"].max()
    route_merge_cols = ["Route", "Direction"]

    all_day_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "All Day")][
        route_merge_cols
        + [
            "Average Scheduled Service (trip minutes)",
            "Average Stop Distance (miles)",
            "# scheduled trips",
            "GTFS Availability",
        ]
    ]

    peak_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "Peak")][
        route_merge_cols + ["Speed (MPH)", "# scheduled trips", "Trips per Hour"]
    ].rename(
        columns={
            "Speed (MPH)": "peak_avg_speed",
            "# scheduled trips": "peak_scheduled_trips",
            "Trips per Hour": "peak_hourly_freq",
        }
    )

    offpeak_stats = df[(df["Date"] == most_recent_date) & (df["Period"] == "Offpeak")][
        route_merge_cols + ["Speed (MPH)", "# scheduled trips", "Trips per Hour"]
    ].rename(
        columns={
            "Speed (MPH)": "offpeak_avg_speed",
            "# scheduled trips": "offpeak_scheduled_trips",
            "frequency": "offpeak_hourly_freq",
        }
    )

    table_df = (
        pd.merge(all_day_stats, peak_stats, on=route_merge_cols, how="outer")
        .merge(offpeak_stats, on=route_merge_cols, how="outer")
        .sort_values(["Route", "Direction"])
        .reset_index(drop=True)
    )

    numeric_cols = table_df.select_dtypes(include="number").columns
    table_df[numeric_cols] = table_df[numeric_cols].fillna(0)
    table_df.columns = table_df.columns.str.title().str.replace("_", " ")
    return table_df

### Average Scheduled Minutes

In [ ]:
def grouped_bar_chart(
    df: pd.DataFrame,
    color_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        color_col,
        y_col,
    ]

    df = section2.clean_data_charts(df, y_col)

    chart = (
        alt.Chart(df)
        .mark_bar(size=10)
        .encode(
            x=alt.X(
                "yearmonthdate(Date):O",
                title=["Grouped by Direction ID", "Date"],
                axis=alt.Axis(labelAngle=-45, format="%b %Y"),
            ),
            y=alt.Y(f"{y_col}:Q", title=_report_utils.labeling(y_col)),
            xOffset=alt.X(f"{offset_col}:N", title=_report_utils.labeling(offset_col)),
            color=alt.Color(
                f"{color_col}:N",
                title=_report_utils.labeling(color_col),
                scale=alt.Scale(
                    range=_report_utils.red_green_yellow,
                ),
            ),
            tooltip=tooltip_cols,
        )
    )
    chart = (chart).properties(
        title={
            "text": [title],
            "subtitle": [subtitle],
        },
        width=500,
        height=300,
    )

    return chart

### Timeliness

In [ ]:
def base_facet_chart(
    df: pd.DataFrame,
    y_col: str,
    color_col: str,
    facet_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        y_col,
        color_col,
    ]

    max_y = section2.set_y_axis(df, y_col)
    df = section2.clean_data_charts(df, y_col)
    chart = (
        (
            alt.Chart(df)
            .mark_bar(size=15, clip=True)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=["Date"],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=tooltip_cols,
            )
        )
        .facet(
            column=alt.Column(
                f"{facet_col}:N",
            )
        )
        .properties(
            title={
                "text": title,
                "subtitle": subtitle,
            }
        )
    )
    return chart

In [ ]:
avg_schedule = grouped_bar_chart(
    df=all_day,
    color_col="Direction",
    y_col="Average Scheduled Service (trip minutes)",
    offset_col="Direction",
    title=readable_dict["avg_scheduled_min_graph"]["title"],
    subtitle=readable_dict["avg_scheduled_min_graph"]["subtitle"],
)

In [ ]:
avg_schedule

In [ ]:
timeliness_trips_dir_1 = base_facet_chart(
    timeliness_df.loc[timeliness_df["Direction"] == 1],
    "value",
    "variable",
    "Period",
    readable_dict["timeliness_trips_dir_1_graph"]["title"],
    readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
)

### Frequency

In [ ]:
def frequency_chart(df: pd.DataFrame):
    df["Frequency in Minutes"] = (
        "A trip going this direction comes every "
        + df.frequency_in_minutes.astype(int).astype(str)
        + " minutes"
    )
    chart = (
        alt.Chart(df)
        .properties(width=180, height=alt.Step(10))
        .mark_bar()
        .encode(
            alt.Y(
                "yearmonthdate(Date):O",
                title="Date",
                axis=alt.Axis(format="%b %Y"),
            ),
            alt.X(
                "frequency_in_minutes:Q",
                title=_report_utils.labeling("frequency_in_minutes"),
                axis=None,
            ),
            alt.Color(
                "frequency_in_minutes:Q",
                scale=alt.Scale(range=_report_utils.green_red_yellow),
            ).title(_report_utils.labeling("frequency_in_minutes")),
            alt.Row("Period:N")
            .title(_report_utils.labeling("Period"))
            .header(labelAngle=0),
            alt.Column("Direction:N").title(_report_utils.labeling("Direction")),
            tooltip=["Date", "Frequency in Minutes", "Period", "Direction"],
        )
    )
    chart = chart.properties(
        title={
            "text": readable_dict["frequency_graph"]["title"],
            "subtitle": readable_dict["frequency_graph"]["subtitle"],
        }
    )
    return chart

In [ ]:
frequency_altair_chart = frequency_chart(df)

### Speed

In [ ]:
def base_facet_line(
    df: pd.DataFrame, y_col: str, title: str, subtitle: str
) -> alt.Chart:
    max_y = section2.set_y_axis(df, y_col)

    df = section2.clean_data_charts(df, y_col)
    tooltip_cols = [
            "Route",
            "Direction",
            "Period",
            f"{y_col}_str",
        ]

    chart = (
            alt.Chart(df)
            .mark_line(size=5)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title="Date",
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    "Period:N",
                    title=_report_utils.labeling("Period"),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=tooltip_cols,
            )
        )

    chart = chart.properties(width=250, height=300)
    chart = chart.facet(
            column=alt.Column("Direction:N", title=_report_utils.labeling("Direction")),
        ).properties(
            title={
                "text": [title],
                "subtitle": [subtitle],
            }
        )
    return chart

In [ ]:
speed_chart = base_facet_line(
    df,
    "Speed (MPH)",
    readable_dict["speed_graph"]["title"],
    readable_dict["speed_graph"]["subtitle"],
)

###  VP per Minute 

In [ ]:
def base_facet_with_ruler_chart(
    df: pd.DataFrame,
    y_col: str,
    ruler_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Organization",
        "Date",
        y_col,
    ]

    max_y = section2.set_y_axis(df, y_col)
    df = section2.clean_data_charts(df, y_col)
    ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"mean({ruler_col}):Q")
        )
    chart = (
            alt.Chart(df)
            .mark_bar(size=15, clip=True)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=["Date"],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=df[tooltip_cols].columns.tolist(),
            )
        )

    chart = chart + ruler
    chart = chart.facet(column=alt.Column("Direction:N",)).properties(
            title={
                "text": title,
                "subtitle": [subtitle],
            }
        )

    return chart

In [ ]:
vp_min = base_facet_with_ruler_chart(
    all_day,
    "Average VP per Minute",
    "ruler_for_vp_per_min",
    readable_dict["vp_per_min_graph"]["title"],
    readable_dict["vp_per_min_graph"]["subtitle"],
)

### Spatial Accuracy

In [ ]:
spatial_accuracy = base_facet_with_ruler_chart(
    all_day,
    "% VP within Scheduled Shape",
    "ruler_100_pct",
    readable_dict["spatial_accuracy_graph"]["title"],
    readable_dict["spatial_accuracy_graph"]["title"],
)

### % VP

In [ ]:
rt_pct = section2.pct_vp_journey(
    all_day,
    "% Actual Trip Minutes with 1+ VP per Minute",
    "% Actual Trip Minutes with 2+ VP per Minute",
)

In [ ]:
rt_pct

In [ ]:
def base_facet_circle(
    df: pd.DataFrame,
    y_col: str,
    color_col: str,
    ruler_col: str,
    title: str,
    subtitle: str,
) -> alt.Chart:

    tooltip_cols = [
        "Direction",
        "Period",
        "Route",
        "Date",
        f"{y_col}_str",
        color_col,
    ]

    max_y = section2.set_y_axis(df, y_col)
    df = section2.clean_data_charts(df, y_col)
    ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"ruler_100_pct:Q")
        )

    chart = (
            alt.Chart(df)
            .mark_circle(size=150)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title="Date",
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(range=_report_utils.red_green_yellow),
                ),
                tooltip=tooltip_cols,
            )
        )

    chart = chart + ruler
    chart = chart.facet(
            column=alt.Column("Direction:N", title=_report_utils.labeling("Direction")),
        ).properties(
            title={
                "text": [title],
                "subtitle": [subtitle],
            }
        )
    return chart

In [ ]:
rt_pct_chart = base_facet_circle(
    rt_pct,
    "% of Actual Trip Minutes",
    "Category",
    "ruler_100_pct",
    readable_dict["rt_vp_per_min_graph"]["title"],
    readable_dict["rt_vp_per_min_graph"]["subtitle"],
)

## Combining #1

In [ ]:
def filtered_route(
    df: pd.DataFrame,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    # Create dropdown
    routes_list = df["Route"].unique().tolist()

    route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes",
    )
    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=["Route"],
        bind=route_dropdown,
    )

    # Filter for only rows categorized as found in schedule and vp and all_day
    all_day = df.loc[df["Period"] == "All Day"].reset_index(drop=True)

    # Create route stats table for the text tables
    route_stats_df = section2.route_stats(df)

    # Manipulate the df for some of the metrics
    timeliness_df = section2.timeliness_trips(df)

    rt_journey_vp = section2.pct_vp_journey(
        all_day,
        "% Actual Trip Minutes with 1+ VP per Minute",
        "% Actual Trip Minutes with 2+ VP per Minute",
    )
    sched_journey_vp = section2.pct_vp_journey(
        all_day,
        "% Scheduled Trip Minutes with 1+ VP per Minute",
        "% Scheduled Trip Minutes with 2+ VP per Minute",
    )

    avg_scheduled_min_graph = (
        grouped_bar_chart(
            df=all_day,
            color_col="Direction",
            y_col="Average Scheduled Service (trip minutes)",
            offset_col="Direction",
            title=readable_dict["avg_scheduled_min_graph"]["title"],
            subtitle=readable_dict["avg_scheduled_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(avg_scheduled_min_graph)
    timeliness_trips_dir_0 = (
        (
            base_facet_chart(
                timeliness_df.loc[timeliness_df["Direction"] == 0],
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_dir_0_graph"]["title"],
                readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
            )
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(timeliness_trips_dir_0)
    timeliness_trips_dir_1 = (
        (
            base_facet_chart(
                timeliness_df.loc[timeliness_df["Direction"] == 1],
                "value",
                "variable",
                "Period",
                readable_dict["timeliness_trips_dir_1_graph"]["title"],
                readable_dict["timeliness_trips_dir_0_graph"]["subtitle"],
            )
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(timeliness_trips_dir_1)
    frequency_graph = (
        frequency_chart(df).add_params(route_selector).transform_filter(route_selector)
    )
    # display(frequency_graph)
    speed_graph = (
        base_facet_line(
            df,
            "Speed (MPH)",
            readable_dict["speed_graph"]["title"],
            readable_dict["speed_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(speed_graph)
    vp_per_min_graph = (
        (
            base_facet_with_ruler_chart(
                all_day,
                "Average VP per Minute",
                "ruler_for_vp_per_min",
                readable_dict["vp_per_min_graph"]["title"],
                readable_dict["vp_per_min_graph"]["subtitle"],
            )
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(vp_per_min_graph)
    rt_vp_per_min_graph = (
        base_facet_circle(
            rt_journey_vp,
            "% of Actual Trip Minutes",
            "Category",
            "ruler_100_pct",
            readable_dict["rt_vp_per_min_graph"]["title"],
            readable_dict["rt_vp_per_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(rt_vp_per_min_graph)
    sched_vp_per_min = (
        base_facet_circle(
            sched_journey_vp,
            "% of Actual Trip Minutes",
            "Category",
            "ruler_100_pct",
            readable_dict["sched_vp_per_min_graph"]["title"],
            readable_dict["rt_vp_per_min_graph"]["subtitle"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(sched_vp_per_min)
    spatial_accuracy = (
        base_facet_with_ruler_chart(
            all_day,
            "% VP within Scheduled Shape",
            "ruler_100_pct",
            readable_dict["spatial_accuracy_graph"]["title"],
            readable_dict["spatial_accuracy_graph"]["title"],
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(spatial_accuracy)
    text_dir0 = (
        (create_text_table(route_stats_df, 0))
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(text_dir0)
    text_dir1 = (
        create_text_table(route_stats_df, 1)
        .add_params(route_selector)
        .transform_filter(route_selector)
    )
    # display(text_dir1)

    chart_list = [
        avg_scheduled_min_graph,
        timeliness_trips_dir_0,
        timeliness_trips_dir_1,
        frequency_graph,
        speed_graph,
        vp_per_min_graph,
        rt_vp_per_min_graph,
        sched_vp_per_min,
        spatial_accuracy,
        text_dir0,
        text_dir1,
    ]

    chart = alt.vconcat(*chart_list)

    return chart

## Combining #2

In [ ]:
stop

In [ ]:
routes_list = df["Route"].unique().tolist()

route_dropdown = alt.binding_select(
    options=routes_list,
    name="Routes",
)
# Column that controls the bar charts
route_selector = alt.selection_point(
    fields=["Route"],
    bind=route_dropdown,
)

In [ ]:
timeliness_trips_dir_1 = timeliness_trips_dir_1.add_params(
    route_selector
).transform_filter(route_selector)

In [ ]:
avg_schedule = avg_schedule.add_params(route_selector).transform_filter(route_selector)

In [ ]:
frequency_altair_chart = frequency_altair_chart.add_params(
    route_selector
).transform_filter(route_selector)

In [ ]:
speed_chart = speed_chart.add_params(route_selector).transform_filter(route_selector)

In [ ]:
vp_min = vp_min.add_params(route_selector).transform_filter(route_selector)

In [ ]:
spatial_accuracy = spatial_accuracy.add_params(route_selector).transform_filter(route_selector)

In [ ]:
rt_pct_chart = rt_pct_chart.add_params(route_selector).transform_filter(route_selector)

In [ ]:
chart_list = [avg_schedule, timeliness_trips_dir_1, frequency_altair_chart,speed_chart, vp_min,spatial_accuracy,rt_pct_chart]

In [ ]:
chart = alt.vconcat(*chart_list).properties(
    resolve=alt.Resolve(
        scale=alt.LegendResolveMap(color=alt.ResolveMode("independent"))
    )
)

In [ ]:
chart